In [1]:
!pip install wandb
!pip install import-ipynb

In [2]:
# Imports
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import wandb
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd '/content/gdrive/MyDrive/deepsf/code_JS'
import import_ipynb

# do_plots_model_results
from do_plots_model_results import val_loss_vs_epoch, ReturnPlotSolution, plot_pred_vs_real, corr_vs_biotype

# class_models_functions
from class_models_pytorch import evaluate, do_training, fit, DeepSF, DeepSF_2hidden 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1H6MhDWUnnQQrC4_3ci_6saOGAZ1koHG4/deepsf/code_JS
importing Jupyter notebook from do_plots_model_results.ipynb
importing Jupyter notebook from class_models_pytorch.ipynb


In [4]:
path = '/content/gdrive/MyDrive/deepsf/code_JS/folder_rawdata_processing/'

# Read the data from the file.
with open(path+'3-pipeline_files.pkl', 'rb') as fid:
     result = pickle.load(fid)

TCGA_tpm_gn_RBPs = result['TCGA_tpm_gn_RBPs'] #pacientes x gene_expression SFs
TCGA_tpm_gn = result['TCGA_tpm_gn'] #pacientes x gene_expression total genes
TCGA_tpm_without_uniqueiso = result['TCGA_tpm_without_uniqueiso'] #pacientes x isoform_expression
getBM = result['getBM'] # index x (Transcript_ID, Gene_ID, Transcrip_name, Gene_name, Biotype)

TCGA_tpm_without_uniqueiso_log2p = np.log2(1+TCGA_tpm_without_uniqueiso) # Transformación Log2 a la matriz de isoformas.
getBM = getBM.iloc[[a in TCGA_tpm_without_uniqueiso_log2p.columns for a in getBM.Transcript_ID], :] # getBM real.

In [19]:
# Hyperparameters
batch_size=32
learning_rate=1e-1
if_wandb = False
test_size = 0.2
num_genes = 10

In [6]:
toy_genes = list(getBM['Gene_name'][:num_genes]) #+ [ 'TP53'] 
getBM = getBM.iloc[[a in toy_genes for a in getBM.Gene_name], :] #getBM reducido.
getBM
  
toy_Transcript_ID = list(getBM.Transcript_ID)
TCGA_tpm_without_uniqueiso_log2p = TCGA_tpm_without_uniqueiso_log2p.loc[:, toy_Transcript_ID] 

  # Creamos el input 2: pacientes x expresión de los genes de cada una de las isoformas (de la matriz de genes total).
TCGA_tpm_gn_expr_each_iso = pd.DataFrame(np.zeros((TCGA_tpm_gn.shape[0], TCGA_tpm_without_uniqueiso_log2p.shape[1])), 
                                         index = TCGA_tpm_gn.index, columns = list(getBM.Gene_name))

for i in list(getBM.Gene_name): 
  TCGA_tpm_gn_expr_each_iso[i] = TCGA_tpm_gn.loc[:,i]
TCGA_tpm_gn_expr_each_iso.head() # pacientes x expresión de los genes de cada una de las isoformas.

  # Split in Training and Validation and Standarization of SFs expression 
df_train, df_validation = train_test_split(TCGA_tpm_gn_RBPs, test_size= test_size, random_state=0)

  # labels (we need the same patients so we use the same index selection)
train_labels = TCGA_tpm_without_uniqueiso_log2p.loc[df_train.index]
valid_labels = TCGA_tpm_without_uniqueiso_log2p.loc[df_validation.index]

  # gen_expr
train_gn =  TCGA_tpm_gn_expr_each_iso.loc[df_train.index]
valid_gn = TCGA_tpm_gn_expr_each_iso.loc[df_validation.index]

  # Scale the SF input data:
scaler_sfs = StandardScaler() #Initialize
scaler_sfs.fit(df_train) #We put the content inside the scaler. For each feature mean and std.

scaledTrain_df = pd.DataFrame(scaler_sfs.transform(df_train),index = df_train.index, columns = df_train.columns)
scaledValidation_df = pd.DataFrame(scaler_sfs.transform(df_validation),index = df_validation.index, columns = df_validation.columns)

  # Scale the gen_expr:
scale_gn = StandardScaler()
scale_gn.fit(train_gn)

scaled_train_gn = pd.DataFrame(scale_gn.transform(train_gn),index = train_gn.index, columns = train_gn.columns)
scaled_valid_gn = pd.DataFrame(scale_gn.transform(valid_gn),index = valid_gn.index, columns = valid_gn.columns)

# Convert to PyTorch dataset
train_ds = TensorDataset(torch.tensor(scaledTrain_df.values, dtype=torch.float32),
                         torch.tensor(train_labels.values, dtype=torch.float32),
                         torch.tensor(scaled_train_gn.values, dtype=torch.float32))

val_ds = TensorDataset(torch.tensor(scaledValidation_df.values, dtype=torch.float32),
                         torch.tensor(valid_labels.values, dtype=torch.float32),
                         torch.tensor(scaled_valid_gn.values, dtype=torch.float32))

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size*2)

,Transcript_ID,Gene_ID,Transcrip_name,Gene_name,Biotype
0,ENST00000000233,ENSG00000004059,ARF5-001,ARF5,protein_coding
1,ENST00000000412,ENSG00000003056,M6PR-001,M6PR,protein_coding
2,ENST00000000442,ENSG00000173153,ESRRA-002,ESRRA,protein_coding
3,ENST00000001008,ENSG00000004478,FKBP4-001,FKBP4,protein_coding
4,ENST00000001146,ENSG00000003137,CYP26B1-001,CYP26B1,protein_coding
...,...,...,...,...,...
157044,ENST00000568330,ENSG00000003249,DBNDD1-006,DBNDD1,nonsense_mediated_decay
157328,ENST00000568662,ENSG00000003249,DBNDD1-007,DBNDD1,retained_intron
157474,ENST00000568838,ENSG00000003249,DBNDD1-003,DBNDD1,protein_coding
193442,ENST00000623401,ENSG00000003249,DBNDD1-005,DBNDD1,TEC


,ARF5,M6PR,ESRRA,FKBP4,CYP26B1,NDUFAF7,FUCA2,DBNDD1,HS3ST1,SEMA3F,...,M6PR,M6PR,FKBP4,ESRRA,CYP26B1,DBNDD1,DBNDD1,DBNDD1,DBNDD1,FKBP4
TCGA-L9-A444-01A-21R-A24H-07_Tumor,102.328937,58.725039,14.221719,29.011417,1.022726,10.653490,18.837590,7.739579,3.549280,8.799622,...,58.725039,58.725039,29.011417,14.221719,1.022726,7.739579,7.739579,7.739579,7.739579,29.011417
TCGA-MP-A4T9-01A-11R-A24X-07_Tumor,241.245779,98.691949,23.398900,21.131922,1.402699,11.641009,59.797727,26.187366,6.347438,19.504741,...,98.691949,98.691949,21.131922,23.398900,1.402699,26.187366,26.187366,26.187366,26.187366,21.131922
TCGA-MP-A4TC-01A-11R-A24X-07_Tumor,210.712247,70.522624,42.007186,33.264544,2.299748,9.139298,50.913499,19.821721,14.484610,11.111145,...,70.522624,70.522624,33.264544,42.007186,2.299748,19.821721,19.821721,19.821721,19.821721,33.264544
TCGA-MP-A4TA-01A-21R-A24X-07_Tumor,180.411901,69.800990,60.610140,104.036641,1.470630,15.359534,56.859416,30.798644,1.809596,15.822463,...,69.800990,69.800990,104.036641,60.610140,1.470630,30.798644,30.798644,30.798644,30.798644,104.036641
TCGA-L4-A4E5-01A-11R-A24X-07_Tumor,258.803455,64.832595,47.872256,53.290103,0.328478,9.668260,28.493565,28.213904,1.448170,8.545552,...,64.832595,64.832595,53.290103,47.872256,0.328478,28.213904,28.213904,28.213904,28.213904,53.290103


StandardScaler()

StandardScaler()

# Model Linear with gene expression

In [10]:
model_linear = DeepSF(n_inputs=TCGA_tpm_gn_RBPs.shape[1], n_outputs=TCGA_tpm_without_uniqueiso_log2p.shape[1])
history = fit(1000, learning_rate, model_linear, train_loader, val_loader, opt_func=torch.optim.SGD)

# Plot results
val_loss_vs_epoch(history)
solution = plot_pred_vs_real(scaledTrain_df, train_labels, scaled_train_gn, model_linear) # training
print('cor_train:', solution.cor_total)
print('cor_trans_train:', solution.cor_trans)

solution2 = plot_pred_vs_real(scaledValidation_df, valid_labels, scaled_valid_gn, model_linear) # validation
print('cor_val:', solution2.cor_total)
print('cor_trans_val:', solution2.cor_trans)

corr_vs_biotype(getBM, train_labels, solution.cor_trans)
corr_vs_biotype(getBM, train_labels, solution2.cor_trans)

Epoch [0], loss: 4.7025, val_loss: 4.5022
Epoch [1], loss: 3.9691, val_loss: 4.5709
Epoch [2], loss: 3.8091, val_loss: 3.7620
Epoch [3], loss: 3.2771, val_loss: 3.7745
Epoch [4], loss: 2.9984, val_loss: 3.2296
Epoch [5], loss: 2.8614, val_loss: 3.0188
Epoch [6], loss: 2.5356, val_loss: 2.8898
Epoch [7], loss: 2.2903, val_loss: 2.6884
Epoch [8], loss: 2.1294, val_loss: 2.4861
Epoch [9], loss: 1.9550, val_loss: 2.3474
Epoch [10], loss: 1.7733, val_loss: 2.2170
Epoch [11], loss: 1.6491, val_loss: 2.0538
Epoch [12], loss: 1.5166, val_loss: 1.9366
Epoch [13], loss: 1.3816, val_loss: 1.8639
Epoch [14], loss: 1.3114, val_loss: 1.7838
Epoch [15], loss: 1.1942, val_loss: 1.6920
Epoch [16], loss: 1.1213, val_loss: 1.5979
Epoch [17], loss: 1.0586, val_loss: 1.5397
Epoch [18], loss: 0.9502, val_loss: 1.4781
Epoch [19], loss: 0.8727, val_loss: 1.4607
Epoch [20], loss: 0.8397, val_loss: 1.3948
Epoch [21], loss: 0.7469, val_loss: 1.2942
Epoch [22], loss: 0.7205, val_loss: 1.2703
Epoch [23], loss: 0.6

KeyboardInterrupt: ignored

# Model 2hidden layer with gene expression

In [20]:
model_2hidden = DeepSF_2hidden(n_inputs=TCGA_tpm_gn_RBPs.shape[1], n_outputs=TCGA_tpm_without_uniqueiso_log2p.shape[1])
history = fit(1000, learning_rate, model_2hidden, train_loader, val_loader, opt_func=torch.optim.SGD)

# Plot the results
val_loss_vs_epoch(history)

solution_2hidden = plot_pred_vs_real(scaledTrain_df, train_labels, scaled_train_gn, model_2hidden) # training
print('cor_train:', solution_2hidden.cor_total)
print('cor_trans_train:', solution_2hidden.cor_trans)

solution_val_2hidden = plot_pred_vs_real(scaledValidation_df, valid_labels, scaled_valid_gn, model_2hidden) # validation
print('cor_val:', solution_val_2hidden.cor_total)
print('cor_trans_val:', solution_val_2hidden.cor_trans)

corr_vs_biotype(getBM, train_labels, solution_2hidden.cor_trans)
corr_vs_biotype(getBM, train_labels, solution_val_2hidden.cor_trans)

Epoch [0], loss: 3.2559, val_loss: 2.5357
Epoch [1], loss: 1.6673, val_loss: 1.3150
Epoch [2], loss: 0.9647, val_loss: 1.1725
Epoch [3], loss: 0.7573, val_loss: 1.1104
Epoch [4], loss: 0.6833, val_loss: 1.0417
Epoch [5], loss: 0.5891, val_loss: 0.9669
Epoch [6], loss: 0.5388, val_loss: 0.8879
Epoch [7], loss: 0.4990, val_loss: 0.8549
Epoch [8], loss: 0.4708, val_loss: 0.8266
Epoch [9], loss: 0.4417, val_loss: 0.7957
Epoch [10], loss: 0.4168, val_loss: 0.7596
Epoch [11], loss: 0.3951, val_loss: 0.7570
Epoch [12], loss: 0.3770, val_loss: 0.7303
Epoch [13], loss: 0.3555, val_loss: 0.6871
Epoch [14], loss: 0.3385, val_loss: 0.6766
Epoch [15], loss: 0.3246, val_loss: 0.6528
Epoch [16], loss: 0.3104, val_loss: 0.6641
Epoch [17], loss: 0.2970, val_loss: 0.6365
Epoch [18], loss: 0.2852, val_loss: 0.6150
Epoch [19], loss: 0.2746, val_loss: 0.6060
Epoch [20], loss: 0.2656, val_loss: 0.6121
Epoch [21], loss: 0.2568, val_loss: 0.5901
Epoch [22], loss: 0.2497, val_loss: 0.5877
Epoch [23], loss: 0.2

KeyboardInterrupt: ignored

In [ ]:
# Boxplot comparar modelos:
corr_values = np.concatenate((solution2.cor_trans, solution_val_2hidden.cor_trans), axis=0)
model_name = np.concatenate((['Model_Linear' for i in solution2.cor_trans], ['Model_2hidden' for i in solution_val_2hidden.cor_trans]), axis=0)

data =  {'model':model_name,
           'corr': corr_values}
         
data_df = pd.DataFrame(data)
ax = sns.boxplot(x="model", y="corr",
                 data=data_df, palette="Set3")

# Optimization of parameters with wandb

In [23]:
learning_rate = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
epochs = 3000
optim = ['sgd90', 'sgd70', 'sgd50', 'adam', 'adagrad', 'adadelta', 'adamW',  'adamax', 'RMSProp']

In [25]:
from torch.optim import optimizer
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
wandb.login()
  
def build_optimizer(network, optimizer, learning_rate): #LEER SOBRE LOS OPTIMIZERS.
  if optimizer == 'sgd90':
    optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=0.9)
  elif optimizer == 'sgd70':
    optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=0.7)
  elif optimizer == 'sgd50':
    optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=0.5)
  elif optimizer == 'adam':
    optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
  elif optimizer == 'adagrad':
    optimizer = torch.optim.Adagrad(network.parameters(), lr=learning_rate)
  elif optimizer == 'adadelta':
    optimizer = torch.optim.Adadelta(network.parameters(), lr=learning_rate)
  elif optimizer == 'adamW':
    optimizer = torch.optim.AdamW(network.parameters(), lr=learning_rate) 
  elif optimizer == 'adamax':
    optimizer = torch.optim.Adamax(network.parameters(), lr=learning_rate) 
  elif optimizer == 'RMSProp':
    optimizer = torch.optim.RMSProp(network.parameters(), lr=learning_rate)
  return optimizer

def wandb_train(model, train_loader, val_loader, hyperparameters, name_project):
  # Tell wandb to get started
  with wandb.init(project=name_project, entity="deepsf", config = hyperparameters):
    config = wandb.config
    wandb.watch(model, criterion = F.mse_loss, log="all", log_freq=10)
    optimizer = build_optimizer(model, config.optimizer, config.learning_rate)

    for epoch in range(config.epochs):
      # Training Phase 
      loss_train_epoch_end = do_training(model, train_loader, optimizer)
      # Validation phase
      loss_val_epoch_end = evaluate(model, val_loader)
      model.epoch_end(epoch, loss_train_epoch_end, loss_val_epoch_end)
       
      wandb.log({"loss": loss_train_epoch_end, "loss_val": loss_val_epoch_end, "epoch": epoch}) 
      
    # Plot results
    solution = plot_pred_vs_real(scaledTrain_df, train_labels, scaled_train_gn, model_linear) # training
    solution2 = plot_pred_vs_real(scaledValidation_df, valid_labels, scaled_valid_gn, model_linear) # validation
    
    wandb.log({"cor_train:": solution.cor_total, "cor_trans_train:": solution.cor_trans})
    wandb.log({"cor_val": solution2.cor_total, "cor_trans_val": solution2.cor_trans})
    wandb.log({"corr_vs_biotype": corr_vs_biotype(getBM, train_labels, solution.cor_trans)})
    wandb.log({"corr_vs_biotype": corr_vs_biotype(getBM, train_labels, solution2.cor_trans)}) 

  # Save the model in the exchangeable ONNX format
  torch.onnx.export(model, "model.onnx")
  wandb.save("model.onnx")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [26]:
for i in optim:
  for j in learning_rate:
    config = dict(
      epochs = epochs,
      learning_rate=j, 
      optimizer=i,
      num_genes=num_genes)
    # Optimization with wandb   
    model_2hidden = DeepSF(n_inputs=TCGA_tpm_gn_RBPs.shape[1], n_outputs=TCGA_tpm_without_uniqueiso_log2p.shape[1])
    wandb_train(model_2hidden, train_loader, val_loader, config, name_project = "try")
                # name_project = "deepsf_version0")

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 951, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 143, in setup
    tel.feature.set_init_tags = True
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/lib/telemetry.py", line 43, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 470, in _telemetry_callback
    self._telemetry_flush()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 481, in _telemetry_flush
    self._backend.interface._publish_telemetry(self._telemetry_obj)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 73, in _publish_telemetry
    self._publish(rec)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_queue.py", line 49, in _publish
    raise Exception("The wa

Exception: ignored

In [35]:
for j in learning_rate:
  print(j)
learning_rate[0]

1
0.1
0.01
0.001
0.0001
1e-05
1e-06
1e-07


1